In [1]:
# imports
from sit1m_data_preprocessing import *
import numpy as np

c:\Users\John\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# download (if not downloaded) and build sift1m dataset
dataset_download_path = "D:/College/SysML/dataset" # a folder to cache dataset contents that are downloaded
splits = build_sift1m(dataset_download_path)

Building Sift1M


Dl Completed...: 0 url [00:00, ? url/s]


Dl Completed...: 100%|██████████| 1/1 [00:01<00:00,  1.44s/ url]
Extraction completed...: 0 file [00:01, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:01<00:00,  1.45s/ url]


In [3]:
# make and print train split input array (1 million embedding arrays of length 128)
train_input_array = get_train_split(splits)

print(train_input_array)
print(train_input_array.shape[1])

Building Train Split Array
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%
[[  0.  16.  35. ...  25.  23.   1.]
 [ 14.  35.  19. ...  11.  21.  33.]
 [  0.   1.   5. ...   4.  23.  10.]
 ...
 [ 30.  12.  12. ...  50.  10.   0.]
 [  0.   5.  12. ...   1.   2.  13.]
 [114.  31.   0. ...  25.  16.   0.]]
128


In [4]:
# make and print test split input array (10k embedding arrays of length 128)
test_input_array, neighbors = get_test_split(splits)

print(test_input_array)
print(test_input_array.shape[1])

Progress: 50.0%
Progress: 100.0%
[[  1.   3.  11. ...  42.  48.  11.]
 [ 40.  25.  11. ...   3.  19.  13.]
 [ 28.   4.   3. ...   2.  54.  47.]
 ...
 [  0.  15.  64. ...   3.  62. 118.]
 [131.   2.   0. ...   7.   0.   0.]
 [ 23.   0.   0. ...  79.  16.   4.]]
128


## PQ Test

In [6]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import faiss

In [7]:
dim = train_input_array.shape[1]
numSubVectors = 8
subVectorBits = 8

pq = faiss.IndexPQ(dim, numSubVectors, subVectorBits)

In [8]:
pq.train(train_input_array)

In [9]:
pq.add(train_input_array)

In [15]:
k = 5
distances, indexes = pq.search(test_input_array[:10000], k)

In [16]:
t = 25
i = 0
relevantPercent = []

for entry in neighbors:

    topIndexs = []
    for x in range(t):
        index_dict = entry[x]
        topIndexs.append(index_dict["index"])

    search_results = indexes[i]

    count = 0

    for x in search_results:
        if x in topIndexs:
            count+=1

    relevantPercent.append(count/k)
    
    i=i+1
        

print(sum(relevantPercent) / len(relevantPercent))

0.5851


## Non-FSQ VQVAE 

In [ ]:
import torch